In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train =pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')
df_test=pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
df_sub=pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
print(df_train['f_27'].nunique())
print(df_test['f_27'].nunique())

In [ ]:
df_train['f_27']

**Feature engineering**

In [ ]:
#New feature for unique characters  in the column f_27
#let=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z'] 
df_train['uniq_chars']= df_train['f_27'].apply(lambda s: len(set(s)))
df_test['uniq_chars']=df_test['f_27'].apply(lambda s: len(set(s)))

In [ ]:
df_train.head()

In [ ]:
from  sklearn.model_selection  import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import matplotlib as plt
import seaborn as sns

In [ ]:
df= df_train.drop(['f_27','id'],axis=1)


In [ ]:
df_test= df_test.drop(['f_27','id'],axis=1)

In [ ]:
df.describe()

In [ ]:

df.nunique()

In [ ]:
float_cols= [col for col in df.columns if df[col].dtype=='float64']
print(float_cols)
cat_cols= [col for col in df.columns if df[col].dtype!='float64' and col!='target']
print(cat_cols)

In [ ]:
# df_sample=df.sample(5000, axis=0) 
# sns.pairplot(df_sample,diag_kind='kde')

****Min and Max for Float features ****

In [ ]:
# df_test['sum_float']= df_test[float_cols].sum(axis=1)
# df_test['std_float']=df_test[float_cols].std(axis=1)
# df_test['avg_float']=df_test[float_cols].mean(axis=1)

# df_train['sum_float']= df_train[float_cols].sum(axis=1)
# df_train['std_float']=df_train[float_cols].std(axis=1)
# df_train['avg_float']=df_train[float_cols].mean(axis=1)

In [ ]:
# df_test['avg_float'].isnull().count()

In [ ]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
for col in float_cols:    
    df[col]  =sc.fit_transform(df[[col]])
    df_test[col]  =sc.transform(df_test[[col]])


In [ ]:
df.describe()

df=pd.getdummies(df, 

In [ ]:
#combine for onehot encoder

Y1=df['target']
X1 =df.drop('target',axis=1)
X1['is_train']=1
df_test['is_train']=0
df_combined = X1.append(df_test)


In [ ]:
df_combined.info()

In [ ]:
df_combined= pd.get_dummies(df_combined, columns=cat_cols)

In [ ]:
df_combined.info()


In [ ]:
X1=df_combined[df_combined['is_train']==1]

In [ ]:
df_test =df_combined[df_combined['is_train']==0]

In [ ]:
X1= X1.drop('is_train',axis=1)
X1.info()

In [ ]:
for col in X1.columns:
 print(col, X1[col].isnull().sum())

In [ ]:
df_test= df_test.drop('is_train',axis=1)
df_test.info()

In [ ]:
from sklearn.model_selection  import KFold

from sklearn.metrics import  roc_auc_score
from xgboost import XGBClassifier

In [ ]:
xgparams={'n_estimators' :8000,
         'learning_rate':0.15,
         'random_state'     : 6,
          'objective'        : 'binary:logistic',
           'tree_method'      : 'gpu_hist'}

In [ ]:
# kf= KFold(n_splits=5)
# score=[]
# Predictions=[]

# for fold,(train_ind,valid_ind) in enumerate(kf.split(X1)):
#     train_X, Valid_X = X1.iloc[train_ind],X1.iloc[valid_ind]
#     train_Y,Valid_Y= Y1.iloc[train_ind],Y1.iloc[valid_ind]
#     model=XGBClassifier(**xgparams)
#     model.fit(train_X,train_Y, eval_set=[(Valid_X,Valid_Y)],eval_metric = ['auc'], early_stopping_rounds = 256, verbose = 0)
#     pred_valid=model.predict(Valid_X)
#     print(pred_valid)
#     roc=roc_auc_score(Valid_Y,pred_valid)
#     score.append(roc)
#     pred_test=model.predict(df_test)
#     Predictions.append(pred_test)
# print(score)

In [ ]:
# df_sub['target'] = np.array(Predictions).mean(axis=0)
# df_sub.to_csv('sumission.csv', index = False)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, models, layers, optimizers
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_auc', mode='max', patience=5, 
                           restore_best_weights=True, verbose=True)
from tensorflow.keras import metrics, regularizers

tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

def  nnModel(inputshape):
    mdl=models.Sequential()
    mdl.add(keras.Input(shape=(inputshape,)))
    mdl.add(layers.Dense(516,activation='relu'))
    mdl.add(layers.Dropout(0.1))
    mdl.add(layers.Dense(256,activation='relu'))
    mdl.add(layers.Dropout(0.1))
    mdl.add(layers.Dense(128,activation='relu'))
    mdl.add(layers.Dropout(0.1))
    mdl.add(layers.Dense(64,activation='relu'))
    mdl.add(layers.Dropout(0.1))
    mdl.add(layers.Dense(1,activation='sigmoid'))
    return mdl

In [ ]:
kf= KFold(n_splits=5)
score2=[]
score=[]
Predictions=[]

for fold,(train_ind,valid_ind) in enumerate(kf.split(X1)):
    train_X, Valid_X = X1.iloc[train_ind],X1.iloc[valid_ind]
    train_Y,Valid_Y= Y1.iloc[train_ind],Y1.iloc[valid_ind]
    print(train_X.shape[1])
    with tpu_strategy.scope():
        model=nnModel(train_X.shape[1])
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                          loss=keras.losses.BinaryCrossentropy(),
                          metrics=[metrics.AUC(name = 'auc')])
            #Fit the model
        history = model.fit(train_X,
                            train_Y,
                            batch_size=4096,
                            epochs=50,
                            validation_data=(Valid_X, Valid_Y),
                            shuffle=True,
                            verbose=1,
                            callbacks=[early_stopping])

#         # plot training history
#         plt.plot(history.history['loss'], label='train')
#         plt.plot(history.history['val_loss'], label='test')
#         plt.legend()
#         plt.show()

        pred_valid=model.predict(Valid_X)
        print(pred_valid)
        roc=roc_auc_score(Valid_Y,pred_valid)
        score2.append(roc)
        pred_test=model.predict(df_test)
        Predictions.append(pred_test)
print(score2)

In [ ]:
df_sub['target'] = np.array(Predictions).mean(axis=0)
df_sub.to_csv('sumission_nn.csv', index = False)